In [1]:
# %pip install seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np
import pygad

In [2]:
def extract_data(file_path):
    ds = pd.read_json(file_path)
    display(ds)
    preferences = np.array([list(row) for row in ds["preferences"]])
    friendships = np.array([list(row) for row in ds["friendships"]])
    authorship = np.array([list(row) for row in ds["authorship"]])
    capacity = ds["reviewer_capacity"][0]
    min_reviews = ds["min_reviews_per_paper"][0]
    max_reviews = ds["max_reviews_per_paper"][0]
    
    return  preferences, friendships, authorship, capacity, min_reviews, max_reviews


In [3]:
def fitness_function_impl(assignment, preferences, capacity, min_reviews, max_reviews, friendships, authorship):
    # Total Preference Score
    preference_score = np.sum(assignment * preferences)
    max_preference_score = np.sum(np.max(preferences, axis=0))  # Max possible score
    normalized_preference_score = preference_score / max_preference_score

    # Capacity Violation
    capacity_violation = np.sum(np.maximum(0, np.sum(assignment, axis=1) - capacity))

    # Review Violations
    reviews_per_paper = np.sum(assignment, axis=0)
    review_violation = np.sum(np.maximum(0, min_reviews - reviews_per_paper)) + \
                       np.sum(np.maximum(0, reviews_per_paper - max_reviews))

    # Friendship Joint Review Violation
    friendship_violation = 0
    for i in range(len(friendships)):
        for j in range(len(friendships)):
            if friendships[i][j] == 1:
                friendship_violation += np.sum(assignment[i] * assignment[j])

    # Friends Reviewing Each Other's Papers
    friends_authorship_violation = 0
    num_reviewers, num_papers = assignment.shape
    for reviewer in range(num_reviewers):
        for paper in range(num_papers):
            for friend in range(num_reviewers):
                if friendships[reviewer][friend] == 1 and authorship[friend][paper] == 1:
                    friends_authorship_violation += assignment[reviewer][paper]

    # Authorship Violation
    authorship_violation = np.sum(assignment * authorship)

    # Weights for penalties
    w1, w2, w3, w4, w5 = 3, 3, 1, 1, 5  # Adjusted weights

    # Fitness
    fitness = normalized_preference_score - (w1 * capacity_violation + w2 * review_violation +
                                             w3 * friendship_violation + w4 * friends_authorship_violation +
                                             w5 * authorship_violation)

    # Bias towards non-zero assignments
    bias_reward = 0.01 * np.sum(assignment)  # Small reward for assignments
    fitness += bias_reward

    return fitness


In [4]:
def generate_valid_solution(num_reviewers, num_papers, reviewer_capacity, min_reviews_per_paper, seed=None):
    rng = np.random.default_rng(seed)
    # Initialize the matrix with zeros
    matrix = np.zeros((num_reviewers, num_papers), dtype=int)
    
    # Fill the matrix ensuring no more than K ones per row
    for i in range(num_reviewers):
        ones_to_place = rng.integers(0, reviewer_capacity + 1)  # Randomly decide how many ones to place (up to K)
        columns = rng.choice(num_papers, ones_to_place, replace=False)  # Randomly select columns
        matrix[i, columns] = 1
    
    # Ensure at least S ones in each column
    for j in range(num_papers):
        current_ones = np.sum(matrix[:, j])
        if current_ones < min_reviews_per_paper:
            additional_ones_needed = min_reviews_per_paper - current_ones
            available_rows = [i for i in range(num_reviewers) if matrix[i, j] == 0 and np.sum(matrix[i, :]) < reviewer_capacity]
            if len(available_rows) < additional_ones_needed:
                continue
            
            # Randomly add ones to meet the column constraint
            rows_to_fill = rng.choice(available_rows, additional_ones_needed, replace=False)
            for row in rows_to_fill:
                matrix[row, j] = 1
    
    return matrix


In [5]:
def generate_random_solution(num_reviewers, num_papers, reviewer_capacity):
    solution = np.zeros((num_reviewers, num_papers), dtype=int)
    for reviewer in range(num_reviewers):
        # Randomly assign up to 'reviewer_capacity' papers to each reviewer
        papers_to_assign = np.random.choice(num_papers, reviewer_capacity, replace=False)
        solution[reviewer, papers_to_assign] = 1
    return solution

In [50]:
def find_solution(file_path):
    preferences, friendships, authorship, capacity, min_reviews, max_reviews = extract_data(file_path)
    num_reviewers, num_papers = preferences.shape

    num_solutions = 50  # Number of solutions in the population

    initial_population = [
        generate_valid_solution(num_reviewers, num_papers, capacity, min_reviews).flatten()
        # generate_random_solution(num_reviewers, num_papers, capacity).flatten()
        for _ in range(num_solutions)
    ]

    def fitness_function(ga_instance, solution, solution_idx):
        # Reshape the 1D solution to a 2D assignment matrix
        assignment = solution.reshape((num_reviewers, num_papers))
    
        # Ensure binary values in the assignment matrix
        assignment = np.clip(assignment, 0, 1)

        # Calculate fitness using the provided fitness function
        return fitness_function_impl(
            assignment,
            preferences,
            capacity,
            min_reviews,
            max_reviews,
            friendships,
            authorship
        )



    ga_instance = pygad.GA(
    num_generations=150,
    num_parents_mating=25,
    fitness_func=fitness_function,
    sol_per_pop=len(initial_population),
    num_genes=num_reviewers * num_papers,  # Flattened assignment matrix
    gene_space=[0,1],
    initial_population=initial_population,  # Custom initial population
    parent_selection_type="tournament",
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=0.1
    )


    # Run the genetic algorithm
    ga_instance.run()

    print()
    solution, solution_fitness, _ = ga_instance.best_solution()
    print("Best solution fitness:", solution_fitness)

    # Reshape the solution into the assignment matrix
    best_assignment = solution.reshape((num_reviewers, num_papers))
    print("Best Assignment Matrix:\n", best_assignment)
    

In [51]:
find_solution("datasets/easy_dataset_1.json")

,dataset_id,num_papers,num_reviewers,reviewer_capacity,min_reviews_per_paper,max_reviews_per_paper,preferences,friendships,authorship
0,Easy Dataset 1,5,5,3,3,5,"[3, 2, 1, 5, 3]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,Easy Dataset 1,5,5,3,3,5,"[5, 2, 4, 2, 2]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]"
2,Easy Dataset 1,5,5,3,3,5,"[4, 3, 1, 2, 4]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,Easy Dataset 1,5,5,3,3,5,"[4, 2, 2, 1, 1]","[0, 0, 0, 0, 0]","[1, 1, 0, 0, 1]"
4,Easy Dataset 1,5,5,3,3,5,"[4, 3, 4, 1, 5]","[0, 0, 0, 0, 0]","[0, 0, 1, 1, 0]"


c:\Users\user\miniconda3\envs\intelsys\Lib\site-packages\pygad\pygad.py:748: UserWarning: The percentage of genes to mutate (mutation_percent_genes=0.1) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resulted in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")
c:\Users\user\miniconda3\envs\intelsys\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting fr


Best solution fitness: -3.7590909090909093
Best Assignment Matrix:
 [[0. 1. 1. 1. 1.]
 [1. 0. 1. 0. 1.]
 [1. 1. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [1. 1. 0. 0. 1.]]
